In [0]:
# Parsing Events for Cyberthon 2020 - 100% Accuracy
# By richdom2185

import pandas as pd

# Open files and read through the lines
with open('logs.txt', encoding='utf-8') as logs:
  logs_read = logs.readlines()
with open('questions.txt', encoding='utf-8') as qns:
  qns__read = qns.readlines()

# Drop the '\n' character at the end of each line
logs_read = [line[:-1] for line in logs_read]
qns__read = [line[:-1] for line in qns__read]

In [0]:
# Convert array of lines to array of dict with standard format for each log item
# when = date + time
# sender = from
# receiver = to
logs_data = []
for (ind, line) in enumerate(logs_read):
  line_split = line.split(' ') # Separates words
  # Do stuff based on the line length (hard-coding)
  if len(line_split) == 8:
    when = line_split[1] + ' ' + line_split[2][:-1] # drop the ',' after the time
    sender = line_split[3]
    receiver = line_split[7][:-1] # drop the '.' after the receiver address
  elif len(line_split) == 10:
    if line_split[3] == 'to':
      when = line_split[8] + ' ' + line_split[9][:-1] # drop the '.'
      sender = line_split[6]
      receiver = line_split[4]
    elif line_split[3] == 'at':
      when = line_split[4] + ' ' + line_split[5]
      if line_split[6] == 'to':
        sender = line_split[9][:-1] # drop the '.'
        receiver = line_split[7]
      elif line_split[6] == 'by':
        sender = line_split[7]
        receiver = line_split[9][:-1] # drop the '.'
      else:
        print(line) # to check that all the possible cases have been accounted for
    else:
      print(line) # to check that all the possible cases have been accounted for
  elif len(line_split) == 9:
    when = line_split[1] + ' ' + line_split[2][:-1] # drop the ',' after the time
    sender = line_split[8][:-1] # drop the '.'
    receiver = line_split[3]
  else:
    print(line) # to check that all the possible cases have been accounted for
  logs_data.append({'line': ind, 'message': line.split('"')[1], 'from' : sender, 'to': receiver, 'time': when})

In [0]:
from pprint import pprint
pprint(logs_data[:10])

In [0]:
for check in logs_data[:10]:
  print(check['from'])

In [0]:
print(logs_data[46141]) # to check random entries + for troubleshooting

In [0]:
# create empty array of len 5000 so that we can track which questions have been answered
# this is in contrast to using answers.append just in case the program is not coded correctly and it picks up more than 1
# or less than 1 answer to each question, messing up the index
answers = ['']*5000

for (ind, line) in enumerate(qns__read):
  line_split = line[:-1].split(' ') # drops '?' at the end of the question and separates into words
  # more hard-coding
  if line_split[0] == 'What':
    if len(line_split) == 7:
      for check in logs_data:
        if check['from'] == line_split[2] and check['time'] == (line_split[5] + ' ' + line_split[6]):
          answers[ind] = check['message']
          #print('found answer {} for question {}'.format(answers[ind], ind))
    elif len(line_split) == 9:
      for check in logs_data:
        if check['from'] == line_split[2] and check['time'] == (line_split[5] + ' ' + line_split[6]) and check['to'] == line_split[8]:
          answers[ind] = check['message']
          #print('found answer {} for question {}'.format(answers[ind], ind))
  elif line_split[0] == 'Who':
    if len(line_split) == 6:
      for check in logs_data:
        if check['from'] == line_split[2] and check['message'] == line_split[4][1:-1]: # drops the '"' surrounding the message
          answers[ind] = check['to']
          #print('found answer {} for question {}'.format(answers[ind], ind))
    elif len(line_split) == 3:
      for check in logs_data:
        if check['message'] == line_split[2][1:-1]: # drops the '"' surrounding the message
          answers[ind] = check['from']
          #print('found answer {} for question {}'.format(answers[ind], ind))
    elif len(line_split) == 10:
      for check in logs_data:
        if check['to'] == line_split[2] and check['time'] == (line_split[8] + ' ' + line_split[9]):
          answers[ind] = check['from']
          #print('found answer {} for question {}'.format(answers[ind], ind))
  else:
      print(line) # checker in case any case is missed
print('Done!')

In [0]:
# Check for unanswered questions
for (a, b) in enumerate(answers):
  if b == '': print(a, end = ' ')
print('end check')

In [0]:
# Convert to CSV
output = pd.DataFrame({'answer': answers})
output.to_csv('events-submission.csv', index=False)

In [0]:
# Submit to CTFSG Grader
import urllib.request, os
urllib.request.urlretrieve('https://raw.githubusercontent.com/alttablabs/ctfsg-utils/master/pyctfsglib.py', './pyctfsglib.py')
print('Downloaded pyctfsglib.py:', 'pyctfsglib.py' in os.listdir())

import pyctfsglib as ctfsg
import random

USER_TOKEN = "REDACTED" # You need to fill this up
GRADER_URL = random.choice([
"http://p7ju6oidw6ayykt9zeglwyxired60yct.ctf.sg:31011/",
"http://p7ju6oidw6ayykt9zeglwyxired60yct.ctf.sg:31012/"
])

grader = ctfsg.DSGraderClient(GRADER_URL, USER_TOKEN)

# Submitting a file
grader.submitFile('./events-submission.csv')